In [ ]:
#dichiarazione di tutte le librerie necessarie per l'esecuzione del codice
!pip install feel_it
import pandas as pd
import networkx.algorithms as nxa
import networkx as nx
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import itertools
import nltk
import wordcloud
from collections import Counter
from nltk.corpus import stopwords
from matplotlib.pyplot import figure
from wordcloud import WordCloud
from feel_it import EmotionClassifier, SentimentClassifier

nltk.download("stopwords")
stop_words = set(stopwords.words("italian")) 
new_stopwords = ["pi", "più","sar","de","et","lor"]
second_stopwrds = set(stopwords.words("english")) 
stop_words = stop_words.union(new_stopwords)
stop_words = stop_words.union(second_stopwrds)


In [ ]:
from google.colab import files #Istruzione che permette di caricare i file da analizzare
uploaded = files.upload()

In [ ]:
tree=ET.parse('Corpus_Tesi.xml')#Blocco che permette di leggere e creare una genialogia sul file XML
root= tree.getroot()
location=root.findall('newsFrom')

In [ ]:
def person_doc(var): #Funzione che viene richiamata nel primo blocco principale, questa ci consente di trovare per ogni newsFrom i personaggi contenuti
  doc_news=var
  frms=doc_news.findall('.//newsFrom')#element.findall cerca tutti gli elementi con un tag che è direttamente figlio dell'elemento corrente
  people_in_doc=[]
  for frm in frms:
     people_in_frm=[]
     x=frm.findall('.//person')
     for p_dc in x:
      people_in_frm.append(p_dc.text)
     people_in_doc.append(people_in_frm)
  return(people_in_doc)

In [ ]:
def funz_edg(var2):#Funzione che richiamata ci permette di creare gli "angoli" del social network
  f_network=var2
  f_network=itertools.combinations(f_network,2)
  for i in f_network:
    edg.append(i)
  return(edg)

In [ ]:
#Blocco principale in cui viene costruito il social network
personaggi_net=person_doc(root)
prsng_tot=[]
for i in personaggi_net:
  prsng_tot.append(list(sorted(set(i)))) 
lista_pers = sorted(set(sum(prsng_tot,[])))
n_pers=len(lista_pers)#Calcolo delle persone nel documento
edg=[]
for i in prsng_tot:
  funz_edg(i)

network_documento=nx.Graph()#Blocco che crea il social network
network_documento.add_nodes_from(lista_pers)
network_documento.add_edges_from(edg)

comp=nx.connected_components(network_documento)#Istruzioni per il calcolo dei cliques e dei components
cliq=nx.find_cliques(network_documento)

print('Personaggi presenti nel documento',n_pers)#Serie di stampa a schermo di valori connessi al network
print('Clique del network:', len(list(cliq)))
print('Densità del network:', nx.density(network_documento))
print('Coefficiente medio di clustering:', nx.average_clustering(network_documento))
print('Componenti connessi:', len(list(comp)))

#Istruzioni della libreria per visualizzare il social network
plt.figure(figsize=(30,25)) 
plt.title("Social network according with documentation",fontsize = 35)
n_pers=nx.spring_layout(network_documento,k=2,scale=1, iterations=70)
nx.draw(network_documento,n_pers,node_color='r')
nx.draw_networkx_labels(network_documento,n_pers)
plt.show()


In [ ]:
print('Network Pagerank del documento selezionato')#Serie di stampe di alcuni valori analizzati relativi al social network
peso=nxa.pagerank(network_documento)#Le istruzioni presenti provengono dalla libreria networkx.algorithms
sorted((v, f"{c:0.2f}") for v, c in peso.items())

In [ ]:
print('Network closeness del documento selezionato')
closeness=nxa.closeness_centrality(network_documento)
sorted((v, f"{c:0.2f}") for v, c in closeness.items())

In [ ]:
print('Diametro Network del documento selezionato')
print(nx.diameter(network_documento))

In [ ]:
print('Network degree del documento selezionato')
degree=nxa.degree_centrality(network_documento)
sorted((v, f"{c:0.2f}") for v, c in degree.items()) 

In [ ]:
#Blocco per estrarre e inserire in liste i luoghi presenti nel documento, i Topics e le persone.
luoghi=[]
ecnript=[]
words=[]
totalizzatore=0
news_totalizzatore=0
personaggi1=[]
argomenti=[]

for item in root.iter('newsFrom'):#Istruzione che scorre il documento alla ricerca dei luoghi
  luoghi.append(item.text)#Istruzione che inserisce l'elemento trovato nella lista
  news_totalizzatore=1+news_totalizzatore
for item in root.iter('encrypted'):#Istruzione che scorre il documento alla ricerca dei Personaggi
  totalizzatore=totalizzatore+1
  ecnript.append(item.text)
for item in root.iter('transc'):words.append(item.text)
for item in root.iter('from'):luoghi.append(item.text)#Istruzione che inserisce l'elemento trovato nella lista
for item in root.iter('newsTopic'):argomenti.append(item.text)#Istruzione che inserisce l'elemento trovato nella lista
for item in root.iter('person'):personaggi1.append(item.text)#Istruzione che inserisce l'elemento trovato nella lista

In [ ]:
res = []
res = list(filter(None, words))
normal_words = "".join(res)
wordcloud = WordCloud(stopwords = stop_words, width=2000, height=1000,collocations=True).generate(normal_words)
plt.figure( figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

In [ ]:
words_sent=words
res = []
for val in words_sent:
    if val != None :
        res.append(val)
emo=EmotionClassifier()
emotion_salvini=emo.predict(res)
sent=SentimentClassifier()
sentiment_salvini=sent.predict(res)

In [ ]:
mention_sal_tab=pd.DataFrame({"Sentiment":sentiment_salvini})
mention_sal_tab
senti_sal_tab=pd.DataFrame({"Emotion":emotion_salvini})
senti_sal_tab

In [ ]:
res

In [ ]:
  plt.figure(figsize=(10,7))
  mention_sal_tab["Sentiment"].value_counts()
  mention_sal_tab["Sentiment"].value_counts().plot(kind="pie",autopct='%1.0f%%')
  plt.title("News Sentiment analysis 1648")
  plt.show()

In [ ]:
  plt.figure(figsize=(10,7))
  senti_sal_tab["Emotion"].value_counts()
  senti_sal_tab["Emotion"].value_counts().plot(kind="pie", autopct='%1.0f%%')
  plt.title("News emotion analysis 1648")
  plt.show()

In [ ]:
print("Notizie con la cifra:", totalizzatore)
print("Notizie totali nel corpus:", news_totalizzatore)

In [ ]:
#Blocco per costruire una tabella che ci permette di vedere quante volte un singolo luogo è presente nel documento
df_luoghi=pd.DataFrame(luoghi, columns=['Luoghi']) 
df_luoghi=df_luoghi.groupby(['Luoghi']).size().reset_index(name="Volte")#questa istruzione permette di raggruppare i luoghi in relazione al numero di volte che compaiono
df_luoghi

In [ ]:
#Blocco per costruire una tabella che ci permette di vedere quante volte un singolo Topic è presente nel documento
df_topic=pd.DataFrame(argomenti, columns=['Topics'])#L'istruzione proviene dalla libreria Panda
df_topic=df_topic.groupby(['Topics']).size().reset_index(name="N°")#questa istruzione permette di raggruppare i Topic in relazione al numero di volte che compaiono
df_topic

In [ ]:
#Blocco per costruire una tabella che ci permette di vedere quante volte un singolo Topic è presente nel documento
df_persone=pd.DataFrame(personaggi1, columns=['Personaggi'])
df_persone=df_persone.groupby(['Personaggi']).size().reset_index(name="Volte")#questa istruzione permette di raggruppare i personaggi in relazione al numero di volte che compaiono
df_persone